In [28]:
# Cell: Load Model and Predict on WAV File

# --- Imports (ensure these are run first in your notebook session) ---
import os
import pandas as pd
import numpy as np
import joblib
import librosa
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from imblearn.pipeline import Pipeline as ImbPipeline # Necessary if your saved model is an ImbPipeline

# --- Configuration ---
MODEL_PATH = '../models/best_smote_randomforest_model_randomized.pkl' # <-- IMPORTANT: Update if your model filename is different
TEST_WAV_FILE = '../useful_sleep_sounds/denoised_wiener_test.wav' # <-- IMPORTANT: Update to the name of your test WAV file
FRAME_DURATION_SEC = 1.0 # Must match the duration used during training data creation
AUDIO_CHANNEL_NAME = 'Mic' # Make sure this matches the channel used for features


# --- 1. Load the Trained Model ---
if os.path.exists(MODEL_PATH):
    try:
        loaded_model_pipeline = joblib.load(MODEL_PATH)
        print(f"Model loaded successfully from '{MODEL_PATH}'.")
    except Exception as e:
        print(f"ERROR: Could not load model. Ensure '{MODEL_PATH}' is correct and not corrupted. Error: {e}")
        loaded_model_pipeline = None
else:
    print(f"ERROR: Model file not found at '{MODEL_PATH}'. Please upload it or check the path.")
    loaded_model_pipeline = None


# --- 2. Load and Preprocess the New WAV File ---
if loaded_model_pipeline and os.path.exists(TEST_WAV_FILE):
    print(f"\nLoading and preprocessing audio from '{TEST_WAV_FILE}'...")
    try:
        # Load the audio using librosa (assuming single channel for prediction)
        # Note: If your original EDF had multiple channels and 'Mic' was extracted,
        # ensure this WAV is indeed single-channel mic audio.
        audio_signal, sr = librosa.load(TEST_WAV_FILE, sr=None) # Load at original sample rate
        
        # --- Feature Extraction per frame ---
        all_frames_features = []
        frame_size_samples = int(FRAME_DURATION_SEC * sr)
        num_frames = len(audio_signal) // frame_size_samples
        
        for i in range(num_frames):
            frame_start_sample = i * frame_size_samples
            frame_end_sample = frame_start_sample + frame_size_samples
            frame = audio_signal[frame_start_sample:frame_end_sample]
            
            # --- Feature extraction logic (must match your data generation) ---
            rms = librosa.feature.rms(y=frame).mean()
            zcr = librosa.feature.zero_crossing_rate(y=frame).mean()
            try:
                centroid = librosa.feature.spectral_centroid(y=frame, sr=sr).mean()
            except ZeroDivisionError:
                centroid = 0 # Handle silent frames
            mfccs = librosa.feature.mfcc(y=frame, sr=sr, n_mfcc=13)
            mfccs_mean = mfccs.mean(axis=1)

            features = {
                'rms': rms,
                'zcr': zcr,
                'centroid': centroid,
                # 'bandwidth': librosa.feature.spectral_bandwidth(y=frame, sr=sr).mean(), # Add back other features
                # 'rolloff': librosa.feature.spectral_rolloff(y=frame, sr=sr).mean()
            }
            for j, val in enumerate(mfccs_mean, 1):
                features[f'mfcc_{j}'] = val
            
            all_frames_features.append(features)
        
        if not all_frames_features:
            print("WARNING: No frames processed. Audio file might be too short.")
            X_predict = pd.DataFrame()
        else:
            X_predict = pd.DataFrame(all_frames_features)
            print(f"Extracted features for {len(X_predict)} frames.")

        # --- 3. Make Predictions ---
        if not X_predict.empty:
            # The loaded_model_pipeline handles both scaling and prediction internally
            predictions = loaded_model_pipeline.predict(X_predict)
            probabilities = loaded_model_pipeline.predict_proba(X_predict)[:, 1] # Probability of class 1 (apnea)


    except Exception as e:
        print(f"ERROR during audio loading or processing: {e}")
else:
    print("Cannot proceed with prediction. Model or test WAV file not loaded/found.")

Model loaded successfully from '../models/best_smote_randomforest_model_randomized.pkl'.

Loading and preprocessing audio from '../useful_sleep_sounds/denoised_wiener_test.wav'...
Extracted features for 3600 frames.


In [29]:
# ... (after you get 'probabilities' from loaded_model_pipeline.predict_proba) ...

# --- Now, apply your custom threshold to create new predictions ---
custom_threshold = 0.20 # The threshold you observed
predictions_custom_thresh = (probabilities > custom_threshold).astype(int)

print(f"\n--- Prediction Results (per 1-second frame) with custom threshold {custom_threshold} ---")
results_df_custom = pd.DataFrame({
    'frame_index': range(len(predictions_custom_thresh)),
    'start_time_sec': np.arange(len(predictions_custom_thresh)) * FRAME_DURATION_SEC,
    'end_time_sec': (np.arange(len(predictions_custom_thresh)) + 1) * FRAME_DURATION_SEC,
    'prediction': predictions_custom_thresh, # Use the new predictions here
    'probability_apnea': probabilities
})
print(results_df_custom.head(10)) 

# --- Summarize Apnea Events with the new custom threshold predictions ---
print(f"\n--- Summarizing Predicted Apnea Events (Post-Processing) with custom threshold {custom_threshold} ---")
apnea_events = []
in_apnea = False
apnea_start_time = 0
min_apnea_duration = 10 # seconds, clinical definition (KEEP THIS)

# Loop through the predictions_custom_thresh array
for idx, prediction_val in enumerate(predictions_custom_thresh):
    current_frame_start_time = idx * FRAME_DURATION_SEC
    current_frame_end_time = (idx + 1) * FRAME_DURATION_SEC

    if prediction_val == 1 and not in_apnea:
        in_apnea = True
        apnea_start_time = current_frame_start_time
    elif prediction_val == 0 and in_apnea:
        current_duration = current_frame_start_time - apnea_start_time
        if current_duration >= min_apnea_duration:
            apnea_events.append((apnea_start_time, current_frame_start_time, current_duration))
        in_apnea = False

# Check for apnea at the very end of the file
if in_apnea:
    current_duration = results_df_custom.iloc[-1]['end_time_sec'] - apnea_start_time
    if current_duration >= min_apnea_duration:
        apnea_events.append((apnea_start_time, results_df_custom.iloc[-1]['end_time_sec'], current_duration))

if apnea_events:
    print(f"Found {len(apnea_events)} significant apnea events (>= {min_apnea_duration} seconds):")
    apnea_summary_df = pd.DataFrame(apnea_events, columns=['Start_Sec', 'End_Sec', 'Duration_Sec'])
    print(apnea_summary_df)
else:
    print("No significant apnea events predicted with this threshold.")


--- Prediction Results (per 1-second frame) with custom threshold 0.2 ---
   frame_index  start_time_sec  end_time_sec  prediction  probability_apnea
0            0             0.0           1.0           1           0.320577
1            1             1.0           2.0           1           0.356000
2            2             2.0           3.0           1           0.380000
3            3             3.0           4.0           1           0.308000
4            4             4.0           5.0           1           0.416000
5            5             5.0           6.0           1           0.272000
6            6             6.0           7.0           1           0.420000
7            7             7.0           8.0           1           0.360000
8            8             8.0           9.0           1           0.240000
9            9             9.0          10.0           1           0.248000

--- Summarizing Predicted Apnea Events (Post-Processing) with custom threshold 0.2 ---
F